In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_excel('D:/teadata/区域竞争力/数据文件/区域竞争力.xlsx')
df.sample(5)

,pid,cid,year,teaarea,teapro,pervalue,ratio,gteapro,rteapro,wlteapro,bteapro,comnum,trademark,other1,other2
1040,51,149,2017-01-01,257.12,36462,68.75,18.83,29806.0,36495.0,35963.000000,2281.00,338874,183849,41719,3314.495361
461,37,66,2019-01-01,178.24,9598,16.46,15.24,15620.0,21164.0,18617.797835,3930.00,416633,268476,34485,5082.129883
1080,51,155,2015-01-01,219.93,50060,74.68,26.86,15195.0,17692.0,43124.100000,2586.00,199000,103480,52361,1507.347412
430,37,62,2016-01-01,41.60,4127,1.80,0.00,2384.0,5711.0,2450.000000,2664.00,122230,73914,10539,6571.505371
875,45,126,2013-01-01,59.73,8808,9.55,0.00,4042.0,4276.0,1028.310000,613.37,38810,22651,17034,858.628296


In [3]:
regional = 'cid'
df_rem = df.drop('pid',axis=1)
df_group = df_rem.groupby([regional,'year']).sum()
df_group.reset_index(inplace=True)
df_group.shape

(1491, 14)

In [4]:
STS = StandardScaler()
X =STS.fit_transform(df_group.iloc[:,3:])
pca = PCA(n_components=4)   #降到4维
pca.fit(X)                  #训练
newX=pca.fit_transform(X)   #降维后的数据
print('贡献率',pca.explained_variance_ratio_)  #输出贡献率
# print(f'newX{newX}')   

贡献率 [0.65595657 0.10436949 0.07598456 0.05250272]


In [5]:
Fac_score = newX*pca.explained_variance_ratio_.reshape(1,-1)
T_Fac_score = Fac_score.sum(1)

In [6]:
T_Fac_S = pd.Series(T_Fac_score, name='Fac_score')
df_total = pd.merge(df_group,T_Fac_S,left_index=True,right_index=True)
df_total = df_total.sort_values('year')
df_total.head()

,cid,year,teaarea,teapro,pervalue,ratio,gteapro,rteapro,wlteapro,bteapro,comnum,trademark,other1,other2,Fac_score
0,1,2013-01-01,151.66,29589,31.80,4.00,10533.0,11436.0,16740.15,4042.00,451484,250223,78783,4869.327148,0.825660
182,27,2013-01-01,67.97,22758,29.60,29.50,9125.0,11367.0,15260.80,2883.00,217617,87093,88054,4665.887695,0.329154
511,74,2013-01-01,123.15,37178,21.93,1.39,8579.0,10047.0,5509.11,632.04,70849,43082,28801,1003.953491,-0.793047
980,141,2013-01-01,76.02,21461,18.60,6.70,2094.0,2100.0,5992.30,2152.00,135690,85529,12744,4846.071289,-0.706845
1302,187,2013-01-01,29.45,5400,5.50,1.00,5965.0,6403.0,1740.00,2277.00,62112,31466,14759,3817.578369,-1.142328


In [7]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1491 entries, 0 to 1490
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   cid        1491 non-null   int64         
 1   year       1491 non-null   datetime64[ns]
 2   teaarea    1491 non-null   float64       
 3   teapro     1491 non-null   int64         
 4   pervalue   1491 non-null   float64       
 5   ratio      1491 non-null   float64       
 6   gteapro    1491 non-null   float64       
 7   rteapro    1491 non-null   float64       
 8   wlteapro   1491 non-null   float64       
 9   bteapro    1491 non-null   float64       
 10  comnum     1491 non-null   int64         
 11  trademark  1491 non-null   int64         
 12  other1     1491 non-null   int64         
 13  other2     1491 non-null   float64       
 14  Fac_score  1491 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(5)
memory usage: 186.4 KB


from flask import render_template, request, jsonify
from flask import Blueprint
from flask import Flask
from flask_cors import CORS


sentiments= Blueprint("sentiments", __name__, template_folder='../templates')
@sentiments.route("/test", methods=['POST'])     
def test():
    return 'Hello world'


app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False
CORS(app, resources=r'/*')
@app.route('/')
def hello_world():
    return 'Hello World!'
app.debug = True
app.register_blueprint(sentiments, url_prefix='/sentiments')  



if __name__ == '__main__':
    app.run(host="0.0.0.0",port=9001,debug=True)

In [15]:
df = pd.read_excel('D:/teadata/国际竞争力/数据文件/teatrade2.xls')
df.head()

,region,id,import_value,import_quantity,import_price,export_value,export_quantity,export_price
0,世界,135,0,0,NaN,4289,912,4703.0
1,斯里兰卡,31,0,0,NaN,2573,843,3052.0
2,台北，中文,72,0,0,NaN,1406,630,2232.0
3,印度,112,0,0,NaN,200,105,1905.0
4,肯尼亚,106,0,0,NaN,70,31,2258.0


In [18]:
df['intercom_v'] = (df['export_value'] - df['import_value'])/(df['export_value'] + df['import_value'])
df['intercom_q'] = (df['export_quantity'] - df['import_quantity'])/(df['export_quantity'] + df['import_quantity'])
df.sort_values('export_quantity', ascending=False,inplace=True)
df.head()

,region,id,import_value,import_quantity,import_price,export_value,export_quantity,export_price,intercom_v,intercom_q
24,新西兰,98,1785365,369864,4827.0,177824,35452,5016.0,-0.818842,-0.825065
12,德国,99,5905,2277,2593.0,60213,11040,5454.0,0.821380,0.658031
11,莫桑比克,127,2372,787,3014.0,26169,10397,2517.0,0.833783,0.859263
10,英国,104,4858,188,25840.0,54733,3872,14136.0,0.836955,0.907389
13,马拉维,128,1575,226,6969.0,3606,1868,1930.0,0.392009,0.784145


In [19]:
df.head().max()

region                  马拉维
id                      128
import_value        1785365
import_quantity      369864
import_price          25840
export_value         177824
export_quantity       35452
export_price          14136
intercom_v         0.836955
intercom_q         0.907389
dtype: object